In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import my_library.constants as constants

In [7]:
fit_systematics = pd.read_csv('/work/halld/home/viducic/systematic_errors/max_changes.csv')
# signifigant_cuts = pd.read_csv('/work/halld/home/viducic/systematic_errors/signifigant_cuts.csv')
# signifigant_cuts = signifigant_cuts[signifigant_cuts['cut'].str.contains('kstar')]
cuts_df = pd.read_csv('/work/halld/home/viducic/systematic_errors/largest_mag_cuts.csv')
cuts_df = cuts_df[cuts_df['cut'].str.contains('kstar')]
# signifigant_cuts = signifigant_cuts[signifigant_cuts['cut'].isin(['kinfit_cl', 'ppi', 'kp', 'ksp'])]
# signifigant_cuts = signifigant_cuts[signifigant_cuts['cut'].isin(['kinfit_cl', 'ppi'])]
path_to_fit_result_files = '/work/halld/home/viducic/data/fit_params/{}/tf1_gaus_cross_section_values.csv'
df_pipkmks = pd.read_csv(path_to_fit_result_files.format('pipkmks'))
df_pipkmks['cross_section_percent_error'] = df_pipkmks['cross_section_error']/df_pipkmks['cross_section']
df_pimkpks = pd.read_csv(path_to_fit_result_files.format('pimkpks'))
df_pimkpks['cross_section_percent_error'] = df_pimkpks['cross_section_error']/df_pimkpks['cross_section']


In [8]:
cut_fit_errors = pd.DataFrame(columns=['channel', 'e', 't', 'stat_error', 'fit_error', 'cut_error', 'sys_error', 'total_percent_error'])

In [9]:
grouped = fit_systematics.groupby(['channel', 'e', 't'])
for (channel, e, t), group in grouped:
    if e ==12:
        continue
    # print(group)
    fit_error = group['max_percent_change'].values[0]
    etc_sig_cuts = cuts_df[(cuts_df['channel'] == channel) & (cuts_df['e'] == e) & (cuts_df['t'] == t)]
    cuts = {}
    cuts_grouped = etc_sig_cuts.groupby('cut')
    for cut, cut_group in cuts_grouped:
        cuts[cut] = cut_group['mag_percent_change'].values[0]
    
    cut_error = np.sqrt(sum([x**2 for x in cuts.values()]))
    middle_of_bin = constants.T_CUT_DICT[t][0] + constants.T_WIDTH_DICT[t]/2
    if channel == 'pipkmks':
        stat_error = df_pipkmks[(df_pipkmks['beam_energy'] == e) & (df_pipkmks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
        cs = df_pipkmks[(df_pipkmks['beam_energy'] == e) & (df_pipkmks['t_bin_middle'] == middle_of_bin)]['cross_section'].values[0]
    else:
        stat_error = df_pimkpks[(df_pimkpks['beam_energy'] == e) & (df_pimkpks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
        cs = df_pimkpks[(df_pimkpks['beam_energy'] == e) & (df_pimkpks['t_bin_middle'] == middle_of_bin)]['cross_section'].values[0]
    sys_error = np.sqrt(fit_error**2 + cut_error**2)
    total_percent_error = np.sqrt(stat_error**2 + sys_error**2)
    cut_fit_errors = cut_fit_errors.append({'channel': channel, 'e': e, 't': middle_of_bin, 'stat_error': stat_error, 'fit_error': fit_error, 'cut_error': cut_error, 'sys_error': sys_error, 'total_percent_error': total_percent_error}, ignore_index=True)



In [10]:
# print(cut_fit_errors)
cut_fit_errors.to_csv('/work/halld/home/viducic/systematic_errors/cs_total_error.csv', index=False)